In [ ]:
__copyright__ = (
    "Flensburg University of Applied Sciences, "
    "Europa-Universität Flensburg, Centre for Sustainable Energy Systems, "
    "DLR-Institute for Networked Energy Systems"
)
__license__ = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__author__ = (
    "ulfmueller, lukasol, wolfbunke, mariusves, s3pp, ClaraBuettner, "
    "CarlosEpia, KathiEsterl, fwitte, gnn, pieterhexen, AmeliaNadal"
)

<img src="http://etrago.readthedocs.io/en/latest/_images/etrago_logo.png" alt="HSF" height="200" width="200"  align="center" >


# Introduction to eTraGo

## Important links

* __[eTraGo Source Code](https://github.com/openego/eTraGo)__
* __[eTraGo Documentation](http://etrago.readthedocs.io/)__


## Installation
The current eTraGo version as well as the python packages jupyterlab and contextily are required to use this notebook. Install these with

`pip install eTraGo`

`pip install jupyterlab contextily`

## Import required general and eTraGo specific python packages

In [ ]:
%%capture
# enable jupyter interactive plotting
%matplotlib widget

# import Etrago API class
from etrago import Etrago

# import plotting function
from etrago.tools.plot import plot_carrier
import matplotlib.pyplot as plt

## Define parameters to run eTraGo

In [ ]:
args = {
    # Setup:
    "db": "egon-data",  # database session
    "scn_name": "eGon2035",  # scenario: eGon2035, eGon100RE, eGon2035_lowflex, eGon100RE_lowflex
    "start_snapshot": 1,
    "end_snapshot": 15,
    "gridversion": None,  # Currently not needed
    "branch_capacity_factor": {"HV": 0.5, "eHV": 0.7},  # p.u. branch rating
    "foreign_lines": {
        "carrier": "AC",  # 'DC' for modeling foreign lines as links
        "capacity": "osmTGmod",  # 'osmTGmod', 'tyndp2020', 'ntc_acer' or 'thermal_acer'
    },
    "scn_extension": None,  # None or array of extension scenarios (currently not provided, but needed once new lines from NEP are set up)
    "scn_decommissioning": None,  # None or decommissioning scenario (currently not provided)
    
    # Optimisation and powerflow:
    "method": {  # Choose method and settings for optimization
        "type": "lopf",  # type of optimization, currently only 'lopf'
        "n_iter": 1,  # abort criterion of iterative optimization, 'n_iter' or 'threshold'
        "pyomo": True,  # set if pyomo is used for model building
    },
    "solver": "gurobi",  # glpk, cplex or gurobi
    "solver_options": {
        "BarConvTol": 1.0e-5,
        "FeasibilityTol": 1.0e-5,
        "method": 2,
        "crossover": 0,
        "logFile": "solver_etrago.log",
        "threads": 4,
    },
    "model_formulation": "kirchhoff",  # formulation of the LPF problem (all are equivalent)
    "extendable": {
        "extendable_components": [
            "as_in_db"
        ],  # Array of components to optimize
        "upper_bounds_grid": {  # Set upper bounds for grid expansion
            # lines in Germany
            "grid_max_D": None,  # relative to existing capacity
            "grid_max_abs_D": {  # absolute capacity per voltage level
                "380": {"i": 1020, "wires": 4, "circuits": 4},
                "220": {"i": 1020, "wires": 4, "circuits": 4},
                "110": {"i": 1020, "wires": 4, "circuits": 2},
                "dc": 0,
            },
            # border crossing lines
            "grid_max_foreign": 4,  # relative to existing capacity
            "grid_max_abs_foreign": None,  # absolute capacity per voltage level
        },
    },
    "generator_noise": 789456,  # a small random noise to the marginal costs of each generator in order to prevent an optima plateau
    "extra_functionality": {},  # Choose function name (e.g. "min_renewable_share" or "cross_border_flow") or {}    
    "load_shedding": False,  # helpful when debugging - a very expensive generator is added to each bus 
    "lpfile": False,  # save pyomo's lp file: False or /path/to/lpfile.lp
    "csv_export": "results",  # save results as csv: False or /path/tofolder
    "pf_post_lopf": {
        "active": True,  # choose if a pf should be performed after the lopf
        "add_foreign_lopf": True,  # keep results of lopf for foreign DC-links
        "q_allocation": "p_nom",  # allocate reactive power via 'p_nom' or 'p'
    },
    
    # Spatial complexity reduction and disaggregation:
    "network_clustering_ehv": False,  # clustering of HV buses to EHV buses
    "network_clustering": {
        "active": True,  # choose if clustering is activated
        "method": "kmedoids-dijkstra",  # choose clustering method: kmeans or kmedoids-dijkstra
        "n_clusters_AC": 60,  # total number of resulting AC nodes (DE+foreign)
        "cluster_foreign_AC": False,  # take foreign AC buses into account, True or False
        "method_gas": "kmedoids-dijkstra",  # choose clustering method: kmeans or kmedoids-dijkstra
        "n_clusters_gas": 17,  # total number of resulting CH4 nodes (DE+foreign)
        "cluster_foreign_gas": False,  # take foreign CH4 buses into account, True or False
        "k_elec_busmap": False,  # False or path/to/busmap.csv
        "k_gas_busmap": False,  # False or path/to/ch4_busmap.csv
        "bus_weight_tocsv": None,  # None or path/to/bus_weight.csv
        "bus_weight_fromcsv": None,  # None or path/to/bus_weight.csv
        "gas_weight_tocsv": None,  # None or path/to/gas_bus_weight.csv
        "gas_weight_fromcsv": None,  # None or path/to/gas_bus_weight.csv
        "line_length_factor": 1,  # Factor to multiply distance between new buses for new line lengths
        "remove_stubs": False,  # remove stubs bevore kmeans clustering
        "use_reduced_coordinates": False,  # If True, do not average cluster coordinates
        "random_state": 42,  # random state for replicability of clustering results
        "n_init": 10,  # affects clustering algorithm, only change when neccesary
        "max_iter": 100,  # affects clustering algorithm, only change when neccesary
        "tol": 1e-6,  # affects clustering algorithm, only change when neccesary
        "CPU_cores": 4,  # number of cores used during clustering, "max" for all cores available.
    },
    "sector_coupled_clustering": {
        "active": True,  # choose if clustering is activated
        "carrier_data": {  # select carriers affected by sector coupling
            "central_heat": {
                "base": ["CH4", "AC"],
                "strategy": "simultaneous",  # select strategy to cluster other sectors
            },
        },
    },
    "disaggregation": None,  # None or 'uniform'
    
    # Temporal complexity reduction and disaggregation:
    "snapshot_clustering": {
        "active": False,  # choose if clustering is activated
        "method": "segmentation",  # 'typical_periods' or 'segmentation'
        "extreme_periods": None,  # consideration of extreme timesteps; e.g. 'append'
        "how": "daily",  # type of period - only relevant for 'typical_periods'
        "storage_constraints": "soc_constraints",  # additional constraints for storages  - only relevant for 'typical_periods'
        "n_clusters": 5,  #  number of periods - only relevant for 'typical_periods'
        "n_segments": 5,  # number of segments - only relevant for segmentation
    },
    "skip_snapshots": 5,  # False or number of snapshots to skip
    "temporal_disaggregation": {
        "active": False,  # choose if temporally full complex dispatch optimization should be conducted
        "no_slices": 8,  # number of subproblems optimization is divided into
    },

    # Other
    "comments": None,
}

## Import and export of the network and data structure

The network can either be imported from a local database or from an online repository.

Follow the instructions [here](https://github.com/openego/eTraGo/tree/features/release-0.9.0#input-data) to get the data-base.

In [ ]:
etrago = Etrago(args, json_path=None)
etrago.build_network_from_db()

After importing the network from the database, call `adjust_network` to adjust the network imported from the database according to given input-parameters, e.g. add load shedding, set generator noise, set foreign lines to links.

In [ ]:
%%capture
etrago.adjust_network()

Etrago uses pypsa's data structure:

In [ ]:
# the pypsa network is stored in:
etrago.network

In [ ]:
etrago.network.buses.head()

In [ ]:
etrago.network.buses.carrier.value_counts()

In [ ]:
etrago.plot_carrier(carrier_links=["AC", "DC"])

In [ ]:
etrago.plot_carrier(carrier_links=["CH4"], carrier_buses=["CH4"])

To export and import an Etrago network to csv files, you can do the following:

In [ ]:
path_export = "etrago_network"

# export
etrago.export_to_csv(path_export)

# import
path_import = "etrago_network"
etrago_import = Etrago(csv_folder_name=path_import)

## Spatial clustering

The following arguments define the settings for the spatial clustering:

```
args = {    
    # Spatial complexity reduction and disaggregation:
    "network_clustering_ehv": False,  # clustering of HV buses to EHV buses
    "network_clustering": {
        "active": True,  # choose if clustering is activated
        "method": "kmedoids-dijkstra",  # choose clustering method: kmeans or kmedoids-dijkstra
        "n_clusters_AC": 60,  # total number of resulting AC nodes (DE+foreign)
        "cluster_foreign_AC": False,  # take foreign AC buses into account, True or False
        "exclusion_area": ["Cuxhaven", "Bremerhaven", "Wesermarsch", "Osterholz", "Bremen"],  # False, path to shapefile or list of nuts names of not cluster area
        "method_gas": "kmedoids-dijkstra",  # choose clustering method: kmeans or kmedoids-dijkstra
        "n_clusters_gas": 17,  # total number of resulting CH4 nodes (DE+foreign)
        "cluster_foreign_gas": False,  # take foreign CH4 buses into account, True or False
        "k_elec_busmap": False,  # False or path/to/busmap.csv
        "k_gas_busmap": False,  # False or path/to/ch4_busmap.csv
        "bus_weight_tocsv": None,  # None or path/to/bus_weight.csv
        "bus_weight_fromcsv": None,  # None or path/to/bus_weight.csv
        "gas_weight_tocsv": None,  # None or path/to/gas_bus_weight.csv
        "gas_weight_fromcsv": None,  # None or path/to/gas_bus_weight.csv
        "line_length_factor": 1,  # Factor to multiply distance between new buses for new line lengths
        "remove_stubs": False,  # remove stubs bevore kmeans clustering
        "use_reduced_coordinates": False,  # If True, do not average cluster coordinates
        "random_state": 42,  # random state for replicability of clustering results
        "n_init": 10,  # affects clustering algorithm, only change when neccesary
        "max_iter": 100,  # affects clustering algorithm, only change when neccesary
        "tol": 1e-6,  # affects clustering algorithm, only change when neccesary
        "CPU_cores": 8,  # number of cores used during clustering, "max" for all cores available.
    },
    "sector_coupled_clustering": {
        "active": True,  # choose if clustering is activated
        "carrier_data": {  # select carriers affected by sector coupling
            "central_heat": {
                "base": ["CH4", "AC"],
                "strategy": "simultaneous",  # select strategy to cluster other sectors
            },
        },
    },
```

### EHV clustering

In [ ]:
etrago.args["network_clustering_ehv"] = True
etrago.ehv_clustering()

In [ ]:
etrago.

In [ ]:
etrago.plot_carrier(carrier_links=["AC", "DC"])

### Network clustering

Run clustering of electrical network:

In [ ]:
%%capture
etrago.spatial_clustering()

In [ ]:
etrago.plot_carrier(carrier_links=["AC", "DC"])

In [ ]:
etrago.plot_clusters()

In [ ]:
etrago.plot_carrier(carrier_links=["CH4"], carrier_buses=["CH4"])

Which bus in the original network corresponds to which bus in the clustered network as well as the original network is stored in `etrago.busmap`.

In [ ]:
# A copy of the main element of the network is stored in:
etrago.busmap["orig_network"]

In [ ]:
import pandas as pd
pd.Series(etrago.busmap["busmap"]).head()

Run clustering of the gas network and attached technologies:

In [ ]:
%%capture
etrago.spatial_clustering_gas()

In [ ]:
etrago.plot_carrier(carrier_links=["CH4"], carrier_buses=["CH4"])

In [ ]:
etrago.plot_clusters(carrier="CH4")

In [ ]:
etrago.plot_carrier(carrier_links=["central_resistive_heater", "central_heat_pump"], carrier_buses=["AC", "central_heat"])

## Reduce temporal complexity

Implemented are:

**Downsampling**

* time-based method
* groups of consecutive time steps are represented by one time step
* for each group, one time step is assumed to be representative
* this representative time step is weighted according to the number of time steps in its group

**Segmentation**

* property-based determination of representative time steps
* time steps are divided into a certain number of clusters so that similar time steps belong to the same clusters
* clusters can have different sizes, i.e. represent segments of different length
* only consecutive time steps are placed in the same clusters
* for each cluster, a representative time step is defined and weithed based on the number of assigned time steps

**Typical periods**

* typical periods are identified based on time-dependent attributes
* first, the original time series is divided into time periods of equal length
* then, the time periods are clustered and representative time periods are selected, which are called typical periods
* to model storage behavior correctly, additional constraints are required

In case of 'typical periods' and 'segmentation' all load p_set time series as wenn as all renewables p_max_pu time series are used to determine clusters.

The following arguments define the settings for the temporal complexity reduction:

```
args = {    
    # Temporal complexity reduction and disaggregation:
    "snapshot_clustering": {
        "active": False,  # choose if clustering is activated
        "method": "segmentation",  # 'typical_periods' or 'segmentation'
        "extreme_periods": None,  # consideration of extreme timesteps; e.g. 'append'
        "how": "daily",  # type of period - only relevant for 'typical_periods'
        "storage_constraints": "soc_constraints",  # additional constraints for storages  - only relevant for 'typical_periods'
        "n_clusters": 5,  #  number of periods - only relevant for 'typical_periods'
        "n_segments": 5,  # number of segments - only relevant for 'segmentation'
    },
    "skip_snapshots": 5,  # Downsampling: False or number of snapshots to skip
    "temporal_disaggregation": {
        "active": False,  # choose if temporally full complex dispatch optimization should be conducted
        "no_slices": 8,  # number of subproblems optimization is divided into
    },
}
```

In [ ]:
# 'typical_periods' and 'segmentation' are called by the following function
#etrago.snapshot_clustering()

In [ ]:
etrago.network.loads_t.p_set.sum(axis=1).to_frame("load_p_set").plot(figsize=(8, 3))
plt.tight_layout()

In [ ]:
etrago.network.snapshots

In [ ]:
# run downsampling
etrago.skip_snapshots()
etrago.network.snapshots

In [ ]:
etrago.network.loads_t.p_set.sum(axis=1).to_frame("load_p_set").plot(figsize=(8, 3))
plt.tight_layout()

Weight of each snapshot is given in:

In [ ]:
etrago.network.snapshot_weightings

## Run linear optimal power flow

The following arguments define the settings for the optimisation:

```
args = {
    "method": {  # Choose method and settings for optimization
        "type": "lopf",  # type of optimization, currently only 'lopf'
        "n_iter": 1,  # abort criterion of iterative optimization, 'n_iter' or 'threshold'
        "pyomo": True,  # set if pyomo is used for model building
    },
    "solver": "glpk",  # glpk, cplex or gurobi
    "solver_options": {},
    "model_formulation": "kirchhoff",  # formulation of the LPF problem (all are equivalent)
    "extendable": {
        "extendable_components": [
            "as_in_db"
        ],  # Array of components to optimize
        "upper_bounds_grid": {  # Set upper bounds for grid expansion
            # lines in Germany
            "grid_max_D": None,  # relative to existing capacity
            "grid_max_abs_D": {  # absolute capacity per voltage level
                "380": {"i": 1020, "wires": 4, "circuits": 4},
                "220": {"i": 1020, "wires": 4, "circuits": 4},
                "110": {"i": 1020, "wires": 4, "circuits": 2},
                "dc": 0,
            },
            # border crossing lines
            "grid_max_foreign": 4,  # relative to existing capacity
            "grid_max_abs_foreign": None,  # absolute capacity per voltage level
        },
    },
    "generator_noise": 789456,  # a small random noise to the marginal costs of each generator in order to prevent an optima plateau
    "extra_functionality": {},  # Choose function name (e.g. "min_renewable_share" or "cross_border_flow") or {}    
    "load_shedding": False,  # helpful when debugging - a very expensive generator is added to each bus 
    "lpfile": False,  # save pyomo's lp file: False or /path/to/lpfile.lp
    "csv_export": "results",  # save results as csv: False or /path/tofolder
    "pf_post_lopf": {
        "active": True,  # choose if a pf should be performed after the lopf
        "add_foreign_lopf": True,  # keep results of lopf for foreign DC-links
        "q_allocation": "p_nom",  # allocate reactive power via 'p_nom' or 'p'
    },
}
```

In [ ]:
#path = "before_lopf"
#etrago.export_to_csv(path)
#etrago = Etrago(csv_folder_name=path)

Extendable storage units:

In [ ]:
etrago.network.storage_units[etrago.network.storage_units.p_nom_extendable].loc[
    :, ["p_nom", "p_nom_min", "p_nom_max", "p_nom_extendable", "carrier", "marginal_cost", "capital_cost"]].head()

In [ ]:
etrago.network.storage_units[etrago.network.storage_units.p_nom_extendable].carrier.unique()

In [ ]:
etrago.network.storage_units.carrier.unique()

Extendable stores:

In [ ]:
etrago.network.stores[etrago.network.stores.e_nom_extendable].loc[
    :, ["e_nom", "e_nom_min", "e_nom_max", "e_nom_extendable", "carrier", "marginal_cost", "capital_cost"]].head()

In [ ]:
etrago.network.stores[etrago.network.stores.e_nom_extendable].carrier.unique()

In [ ]:
etrago.network.stores.carrier.unique()

Extendable lines:

In [ ]:
etrago.network.lines[etrago.network.lines.s_nom_extendable].loc[
    :, ["s_nom", "s_nom_min", "s_nom_max", "s_nom_extendable", "carrier", "v_nom", "capital_cost", "country"]].head()

In [ ]:
etrago.lopf()

After the optimisation you can run the following to:

* conduct LOPF with full complex time series for dispatch disaggregation

```
    etrago.dispatch_disaggregation()
```

* run power flow to obtain reactive power flows over lines

```
    etrago.pf_post_lopf()
```

* conduct spatial disaggregation of clustered results

```
    etrago.disaggregation()
```

## Results

In [ ]:
etrago.plot_grid(line_colors="expansion_abs", bus_colors="storage_expansion", bus_sizes= 0.000001)

In [ ]:
etrago.calc_results()
etrago.results

In [ ]:
from etrago.tools.plot import curtailment, nodal_gen_dispatch, flexibility_usage

In [ ]:
nodal_gen_dispatch(etrago.network)

In [ ]:
curtailment(etrago.network, carrier="wind_onshore")

In [ ]:
flexibility_usage(etrago, "heat")